# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [1]:
import pandas as pd

In [2]:
import requests

In [4]:
from bs4 import BeautifulSoup

In [6]:
import time

In [98]:
def get_habr_articles(query):
    habr_articles = pd.DataFrame()
    url = 'https://habr.com/ru/search/'
    params = {
        'q': query
    }
    res = requests.get(url, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text)
    articles = soup.find_all('article', class_='tm-articles-list__item')
    for article in articles:
            date = pd.to_datetime(article.find('time').get('title'))
            title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
            link = article.find('a', class_='tm-article-snippet__title-link').get('href')
            row = {'date': date, 'title': title, 'link': 'https://habr.com' + link}
            habr_articles = pd.concat([habr_articles, pd.DataFrame([row])])
    return habr_articles.drop_duplicates().reset_index(drop=True)

res = get_habr_articles(['python', 'анализ данных'])
res

,date,title,link
0,2021-12-10 13:15:00,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...
1,2017-06-27 13:51:00,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331746/
2,2017-06-30 10:52:00,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331998/
3,2022-06-09 21:30:00,Шаблон новичка на пути PANDAS в искусстве анал...,https://habr.com/ru/post/670668/
4,2021-05-06 06:01:00,"Python, исследование данных и выборы: часть 3",https://habr.com/ru/post/556042/
5,2020-09-22 16:43:00,Напишем и поймем Decision Tree на Python с нул...,https://habr.com/ru/post/520204/
6,2021-10-18 15:08:00,AI Journey Contest 2021: какие задачи мы подго...,https://habr.com/ru/company/sbercloud/news/t/5...
7,2014-06-30 23:06:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/228187/
8,2014-11-23 16:49:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/243967/
9,2015-01-18 18:02:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/248165/


## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```


In [166]:
def get_habr_articles(query, num_of_scrap_page=1):
    habr_articles = pd.DataFrame()
    url = 'https://habr.com/ru/search/'
    params = {
        'q': query
    }
    for i in range(1, num_of_scrap_page+1):
        res = requests.get(url+'page'+str(i), params)
        soup = BeautifulSoup(res.text)
        time.sleep(0.3)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        
        for article in articles:
                date = pd.to_datetime(article.find('time').get('title'))
                title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
                link = article.find('a', class_='tm-article-snippet__title-link').get('href')
                req = requests.get('https://habr.com' + link).text
                soup_full_articles = BeautifulSoup(req)
                time.sleep(0.3)
                full_text = soup_full_articles.find('div', class_=['article-formatted-body article-formatted-body article-formatted-body_version-2', 'article-formatted-body article-formatted-body article-formatted-body_version-1', 'tm-expired-company']).text.strip()
                likes = soup_full_articles.find('span', class_=['tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating', 'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating', 'tm-expired-company']).text.strip(' +')
                row = {'date': date, 'title': title, 'link': 'https://habr.com' + link, 'text': full_text, 'num of likes': likes}
                habr_articles = pd.concat([habr_articles, pd.DataFrame([row])])

    return habr_articles.drop_duplicates().reset_index(drop=True)

res = get_habr_articles(['python', 'анализ данных'], 4)
res

,date,title,link,text,num of likes
0,2021-12-10 13:15:00,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...,"14 декабря в Нетологии пройдёт вебинар, посвящ...",4
1,2017-06-27 13:51:00,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331746/,Если Вы только начинаете свой путь знакомства ...,8
2,2017-06-30 10:52:00,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331998/,Первая часть статьи была опубликована тут.\n\n...,15
3,2022-06-09 21:30:00,Шаблон новичка на пути PANDAS в искусстве анал...,https://habr.com/ru/post/670668/,Доброго времени суток! Меня зовут Алексей. Сей...,5
4,2021-05-06 06:01:00,"Python, исследование данных и выборы: часть 3",https://habr.com/ru/post/556042/,Пост №3 для начинающих посвящен генерированию ...,0
...,...,...,...,...,...
75,2020-05-12 13:52:00,Внутри виртуальной машины Python. Часть 1,https://habr.com/ru/post/501338/,Оглавление\n\n1 часть\n2 часть\n\nВведение\n\n...,28
76,2020-05-19 11:07:00,Как построить диаграмму Венна с 50 кругами? Ви...,https://habr.com/ru/company/yandex/blog/501924/,"Всем привет, меня зовут Фёдор Индукаев, я рабо...",41
77,2020-08-10 13:28:00,Алгоритм ранжирования сегментов речной сети с ...,https://habr.com/ru/post/514526/,"Привет, Хабр. \nВ данной статье хотелось бы за...",5
78,2020-11-03 18:52:00,Развертывание интерактивных визуализаций данны...,https://habr.com/ru/company/skillfactory/blog/...,"Сегодня, в преддверии старта нового потока кур...",18


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
